In [1]:
import pandas as pd
import pyproj
import time
import shapefile as shp
import geopandas as gpd
import seaborn as sns
from read_shp_file import read_shapefile

# I. Load Grids
Load rectangular grids generated by QGIS

In [2]:
# grid_fp = "/home/swang/Desktop/shenghao-repos/asiatique/data/penang_grid_EPSG3857_WGS84_v3.csv"
grid_fp = "/Users/shenghao/Desktop/shenghao-repos/asiatique/data/penang_grid_EPSG3857_WGS84_v3.csv"
grid_df = pd.read_csv(grid_fp)
grid_df["id"] = grid_df["id"].apply(lambda grid_id: str(grid_id))
grid_df = grid_df.set_index("id")
grid_df = grid_df.dropna()
print(grid_df.shape)
grid_df.head()

(1199, 5)


,left,top,right,bottom,district
id,,,,,
13,11151054.94,610925.2518,11152054.94,609925.2518,Barat Daya
14,11151054.94,609925.2518,11152054.94,608925.2518,Barat Daya
16,11151054.94,607925.2518,11152054.94,606925.2518,Barat Daya
17,11151054.94,606925.2518,11152054.94,605925.2518,Barat Daya
18,11151054.94,605925.2518,11152054.94,604925.2518,Barat Daya


Convert WGS84 coordinate system to latitude/longitude

In [3]:
def convert_utm_coords(coords, inProj, outProj):
    lng, lat = pyproj.transform(inProj, outProj, coords[0], coords[1])
    return pd.Series([lng, lat])

In [4]:
inProj = pyproj.Proj(init='epsg:3857')
outProj = pyproj.Proj(init='epsg:4326')
start_time = time.time()
print("Converting UTM coordinates to latitude/longitude ...")
grid_df[["left_lng", "top_lat"]] = grid_df.apply(lambda row: convert_utm_coords(row[["left", "top"]], inProj, outProj), axis=1)
grid_df[["right_lng", "bottom_lat"]] = grid_df.apply(lambda row: convert_utm_coords(row[["right", "bottom"]], inProj, outProj), axis=1)
print("Elapsed time: %s seconds ..." %round(time.time() - start_time, 4))
grid_df.head()

Converting UTM coordinates to latitude/longitude ...
Elapsed time: 68.4937 seconds ...


,left,top,right,bottom,district,left_lng,top_lat,right_lng,bottom_lat
id,,,,,,,,,
13,11151054.94,610925.2518,11152054.94,609925.2518,Barat Daya,100.171631,5.479662,100.180614,5.470720
14,11151054.94,609925.2518,11152054.94,608925.2518,Barat Daya,100.171631,5.470720,100.180614,5.461778
16,11151054.94,607925.2518,11152054.94,606925.2518,Barat Daya,100.171631,5.452835,100.180614,5.443893
17,11151054.94,606925.2518,11152054.94,605925.2518,Barat Daya,100.171631,5.443893,100.180614,5.434950
18,11151054.94,605925.2518,11152054.94,604925.2518,Barat Daya,100.171631,5.434950,100.180614,5.426007


# II. Assign Residential Buildings to Grids

In [5]:
def assign_grid(coords, grid_dict):
    for grid_id, boundaries in grid_dict.items():
        if coords[0] > boundaries["left_lng"] and \
           coords[0] < boundaries["right_lng"] and \
           coords[1] > boundaries["bottom_lat"] and \
           coords[1] < boundaries["top_lat"]:
            return str(grid_id)
    return None

In [7]:
grid_dict = grid_df.to_dict('index')
# buildings_fp = "/home/swang/Desktop/shenghao-repos/asiatique/data/penang_residential_buildings.csv"
buildings_fp = "/Users/shenghao/Desktop/shenghao-repos/asiatique/data/penang_residential_buildings.csv"
buildings_df = pd.read_csv(buildings_fp)
print("Range of longitude: ", buildings_df["center_lng"].min(), buildings_df["center_lng"].max())
print("Range of latitude: ", buildings_df["center_lat"].min(), buildings_df["center_lat"].max())
start_time = time.time()
print("Assign residential building to grids ...")
buildings_df["grid"] = buildings_df.apply(lambda row: assign_grid(row[["center_lng", "center_lat"]], grid_dict), axis=1)
buildings_df = buildings_df.set_index("id")
print("Elapsed time: %s seconds ..." %round(time.time() - start_time, 4))
buildings_df.head()

Range of longitude:  100.19315481666666 100.534263925
Range of latitude:  5.1504309 5.531056700000001
Assign residential building to grids ...
Elapsed time: 33.7274 seconds ...


,name,type,area,center_lng,center_lat,grid
id,,,,,,
0,Forest Field,apartments,941.851250,100.294707,5.435854,706
1,Park Avenue,apartments,1198.202200,100.295230,5.434696,707
2,NaN,apartments,1298.948694,100.286958,5.392957,658
3,NaN,apartments,2041.100327,100.286020,5.393071,658
4,NaN,apartments,1298.708829,100.286121,5.393432,658


# III. Compute Gridwise Total Floor Area
Check out available building types.

In [8]:
print("All building types: ", buildings_df["type"].unique())
buildings_df.groupby(['type'])['area'].agg('sum')

All building types:  ['apartments' 'residential' 'bungalow' 'dormitory' 'detached']


type
apartments     3.437263e+06
bungalow       1.605638e+05
detached       1.305754e+04
dormitory      2.046372e+03
residential    6.333936e+04
Name: area, dtype: float64

In [9]:
def check_bungalow(building_type, area):
    return pd.Series([0, area]) if building_type == 'bungalow' else pd.Series([area, 0])

In [10]:
buildings_df[["area", "area_bungalow"]] = buildings_df.apply(lambda row: check_bungalow(row["type"], row["area"]), axis=1)
print(buildings_df.shape)
buildings_df.head()

(3123, 7)


,name,type,area,center_lng,center_lat,grid,area_bungalow
id,,,,,,,
0,Forest Field,apartments,941.851250,100.294707,5.435854,706,0.0
1,Park Avenue,apartments,1198.202200,100.295230,5.434696,707,0.0
2,NaN,apartments,1298.948694,100.286958,5.392957,658,0.0
3,NaN,apartments,2041.100327,100.286020,5.393071,658,0.0
4,NaN,apartments,1298.708829,100.286121,5.393432,658,0.0


In [11]:
area_df = buildings_df.groupby(['grid'])['area', 'area_bungalow'].agg('sum')
area_df = pd.merge(area_df, grid_df, left_index=True, right_index=True)
# area_df = area_df.drop(["left", "right", "top", "bottom"], axis=1)
area_df = area_df.drop(["left", "right", "top", "bottom",
                        "left_lng", "top_lat", "right_lng", "bottom_lat"], axis=1)
area_df = area_df.reset_index()
print(area_df.shape)
area_df.head()

(230, 4)


,index,area,area_bungalow,district
0,1185,11118.712066,0.0,Seberang Perai Utara
1,1186,21331.108587,0.0,Seberang Perai Utara
2,1187,14614.294470,0.0,Seberang Perai Utara
3,1188,30673.413941,0.0,Seberang Perai Utara
4,1229,2720.818373,0.0,Seberang Perai Utara


# IV. Distribute Penang Population into Grids
Assume population density of bungalows: 5 ppl / bungalow with 100m2 floor area.

In [12]:
district_df = area_df.groupby(["district"])['area', 'area_bungalow'].agg('sum')
district_df["total_population"] = [228000, 191400, 422900, 331900, 572500]
district_df["bungalow_population"] = district_df["total_population"] / 100 * 5
district_df["apartment_population"] = district_df["total_population"] - district_df["bungalow_population"]
district_df = district_df.reset_index()
district_df.head()

,district,area,area_bungalow,total_population,bungalow_population,apartment_population
0,Barat Daya,7.532226e+05,9131.675891,228000,11400.0,216600.0
1,Seberang Perai Selatan,7.712756e+04,2421.788262,191400,9570.0,181830.0
2,Seberang Perai Tengah,5.017823e+05,144586.132263,422900,21145.0,401755.0
3,Seberang Perai Utara,3.198889e+05,421.669749,331900,16595.0,315305.0
4,Timur Laut,1.840338e+06,4002.494545,572500,28625.0,543875.0


In [13]:
population_df = pd.merge(area_df, district_df[["district", "area", "apartment_population"]], on='district')
population_df = population_df.rename(columns={
    "index": "grid_id",
    "area_x": "area",
    "area_bungalow_x": "area_bungalow",
    "area_y": "area_apartment_district",
    "apartment_population": "apartment_population_district"
})
population_df["population"] = population_df["apartment_population_district"] / population_df["area_apartment_district"] * population_df["area"] + \
                            population_df["area_bungalow"] / 100 * 5
population_df["grid_id"] = population_df["grid_id"].apply(lambda grid_id: int(grid_id))
print(population_df.shape)
population_df.head()

(230, 7)


,grid_id,area,area_bungalow,district,area_apartment_district,apartment_population_district,population
0,1185,11118.712066,0.0,Seberang Perai Utara,319888.948964,315305.0,10959.382996
1,1186,21331.108587,0.0,Seberang Perai Utara,319888.948964,315305.0,21025.437780
2,1187,14614.294470,0.0,Seberang Perai Utara,319888.948964,315305.0,14404.874357
3,1188,30673.413941,0.0,Seberang Perai Utara,319888.948964,315305.0,30233.869641
4,1229,2720.818373,0.0,Seberang Perai Utara,319888.948964,315305.0,2681.829553


# V. Incorporate Grid Population with Shape File

In [15]:
grid_shape = "/Users/shenghao/Desktop/shenghao-repos/asiatique/data/penang_grid_EPSG3857_WGS84_v3.shp"
# grid_shape = "/home/swang/Desktop/shenghao-repos/asiatique/data/penang_grid_EPSG3857_WGS84_v3.shp"
sf = shp.Reader(grid_shape)
shp_df = read_shapefile(sf)
print(shp_df.shape)
shp_df.head()

fields:  ['left', 'top', 'right', 'bottom', 'id']
(2279, 6)


,left,top,right,bottom,id,coords
0,1.115105e+07,622925.251813,1.115205e+07,621925.251813,1,"[(11151054.943756538, 622925.2518128775), (111..."
1,1.115105e+07,621925.251813,1.115205e+07,620925.251813,2,"[(11151054.943756538, 621925.2518128775), (111..."
2,1.115105e+07,620925.251813,1.115205e+07,619925.251813,3,"[(11151054.943756538, 620925.2518128775), (111..."
3,1.115105e+07,619925.251813,1.115205e+07,618925.251813,4,"[(11151054.943756538, 619925.2518128775), (111..."
4,1.115105e+07,618925.251813,1.115205e+07,617925.251813,5,"[(11151054.943756538, 618925.2518128775), (111..."


In [18]:
population_shp_df = pd.merge(shp_df, population_df[["grid_id", "population"]],
                             left_on='id', right_on='grid_id', how='outer')
population_shp_df["population"].fillna(0, inplace=True)
population_shp_df = population_shp_df.drop(["grid_id", "coords"], axis=1)
print(population_shp_df.shape)
population_shp_df.to_csv("/Users/shenghao/Desktop/shenghao-repos/asiatique/data/penang_grid_population.csv",
                         index=False)
population_shp_df.head()

(2279, 6)


,left,top,right,bottom,id,population
0,1.115105e+07,622925.251813,1.115205e+07,621925.251813,1,0.0
1,1.115105e+07,621925.251813,1.115205e+07,620925.251813,2,0.0
2,1.115105e+07,620925.251813,1.115205e+07,619925.251813,3,0.0
3,1.115105e+07,619925.251813,1.115205e+07,618925.251813,4,0.0
4,1.115105e+07,618925.251813,1.115205e+07,617925.251813,5,0.0


In [15]:
grid_shape = "/home/swang/Desktop/shenghao-repos/asiatique/data/penang_grid_EPSG3857_WGS84_v3.shp"
population_shape = "/home/swang/Desktop/shenghao-repos/asiatique/data/penang_grid_population.shp"

gdf = gpd.read_file(grid_shape)
gdf = gdf.to_crs({'init': 'epsg:3857'})
gdf['population'] = population_shp_df["population"]
gdf.to_file(population_shape)

In [16]:
pop_sf = shp.Reader(population_shape)
pop_df = read_shapefile(pop_sf)
print(pop_df.shape)
pop_df.head()

fields:  ['left', 'top', 'right', 'bottom', 'id', 'population']
(2279, 7)


,left,top,right,bottom,id,population,coords
0,1.115105e+07,622925.251813,1.115205e+07,621925.251813,1,0.0,"[(11151054.943756538, 622925.2518128775), (111..."
1,1.115105e+07,621925.251813,1.115205e+07,620925.251813,2,0.0,"[(11151054.943756538, 621925.2518128775), (111..."
2,1.115105e+07,620925.251813,1.115205e+07,619925.251813,3,0.0,"[(11151054.943756538, 620925.2518128775), (111..."
3,1.115105e+07,619925.251813,1.115205e+07,618925.251813,4,0.0,"[(11151054.943756538, 619925.2518128775), (111..."
4,1.115105e+07,618925.251813,1.115205e+07,617925.251813,5,0.0,"[(11151054.943756538, 618925.2518128775), (111..."
